In [34]:
from pdfextractor import extractor
import pdfextractor
import pickle
import jieba
import pandas as pd
import re
import requests
import operator

In [222]:
class jmdPDFMap(object):
    """
    根据pdf生成文章，并计算推荐的准确率和召回率
    意图：导语，业绩变动原因，主营业务$业务进展，未来计划
    """
    def __init__(self):
        self.labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
        self.labelmap = {
                         '导语': ['意图_企业业绩_现状偏好', '意图_企业业绩_现状偏坏', '导语', '意图_企业基本信息介绍_企业基本信息介绍', '意图_企业活动_公司公告信息'],
                         '业绩变动原因': ['意图_企业业绩_业绩变动原因'],
                         '主营业务$业务进展': ['意图_企业业务_业务介绍', '意图_企业业务_业务规模', '意图_企业业务_业务规划', '意图_企业活动_人事活动', '意图_企业活动_融资活动','意图_企业活动_投资活动'],
                         '未来计划': ['意图_企业业务_业务前景', '意图_企业业绩_前景偏好','意图_企业业绩_前景偏好'],
                         'other': ['其他', '意图_评论_风险提示','意图_评论_投资建议', '意图_评论_盈利预测']
                        }
        self.labelmapR = {
                          '意图_企业业绩_现状偏好': '导语', '意图_企业业绩_现状偏坏': '导语', '导语': '导语', '意图_企业基本信息介绍_企业基本信息介绍': '导语', 
                          '意图_企业业绩_业绩变动原因': '业绩变动原因',
                          '意图_企业业务_业务介绍': '主营业务$业务进展', '意图_企业业务_业务规模': '主营业务$业务进展', '意图_企业业务_业务规划': '主营业务$业务进展', 
                          '意图_企业业务_业务前景': '未来计划', '意图_企业业绩_前景偏好': '未来计划', '意图_企业业绩_前景偏坏': '未来计划',
                          '意图_企业活动_公司公告信息': '主营业务$业务进展', '意图_企业活动_投资活动': '主营业务$业务进展', '意图_企业活动_人事活动': '主营业务$业务进展', '意图_企业活动_融资活动': '主营业务$业务进展', '其他': 'other', '意图_评论_风险提示': 'other', '意图_评论_投资建议': 'other', '意图_评论_盈利预测':'other'
                         }
        self.prop = [0.2, 0.2, 0.4, 0.3, 0]
        self.K = 0
        #每个意图对应的句子或段落数量
        self.num = {}
        self.recommendList = []
        
        
    def computePR(self, article, textList = None, pdfurl = None, recList = None, mode = 'para'):
        """
        pdfurl和textList为内容来源，必须有且仅有一个为None
        mode 必须指明
        """
        intentList = []
        #article_sen = self.get_article_sentence(article)
        article_sen = article
        article_len = len(article_sen)
        if article_len == 0:
            return -1, -1
        print 'article length: ' + str(article_len)
        if recList is None:
            if not pdfurl is None:
                if mode == 'para':
                    try:
                        pdftext = self.extractPDF(pdfurl, 0.8, ['para'])
                    except BaseException:
                        return -1, -1
                else:
                    try:
                        pdftext = self.extractPDF(pdfurl, 0.8, ['text'])
                    except BaseException:
                        return -1, -1
            else:
                pdftext = textList
            for t in pdftext:
                intentList.append(self.getIntent(t))
            if mode == 'para':
                self.K = int(len(article_sen) * 0.5) + 1
            else:
                self.K = int(len(article_sen) * 1.5) + 1
            for l in range(len(self.labels)):
                if l == len(self.labels) - 1:
                    self.num[self.labels[l]] = 0
                else:
                    self.num[self.labels[l]] = int(self.K * self.prop[l]) + 1
            self.recommendList = self.getTopKintent(intentList)
        else:
            self.recommendList = recList
        rec_len = len(self.recommendList)
        if rec_len == 0:
            return -1, -1
        print 'recommend list length: ' + str(rec_len)
        P_dic = {}
        Pcount = 0
        Rcount = 0
        for r in self.recommendList:
            P_dic[r] = False
        for sen in article_sen:
            for rec in self.recommendList:
                if self.MatchTwoText(sen, rec):
                    Rcount += 1
                    if P_dic[rec] is False:
                        P_dic[rec] = True
                        Pcount += 1
                    break
        P = Pcount * 1.0 / rec_len
        R = Rcount * 1.0 / article_len
        print 'Pcount: ' + str(Pcount)
        print 'Rcount: ' + str(Rcount)
        return P, R
    
    def getRec(self):
        return self.recommendList
        
    def getTopKintent(self, intentlist):
        intent = {}
        for l in self.labels:
            intent[l] = {}
        
        for i in intentlist:
            key = i['intent']
            intent[key][i['content']] = i['prob']
        result = []
        for l in self.labels:
            intent_l = intent[l]
            intent[l] = sorted(intent_l.items(), key=operator.itemgetter(1))
        print "各个段落（句子）的预测意图分布为："
        for i in intent:
            print i + "  " + str(len(intent[i]))
        for l in intent.keys():
            if self.num[l] == 0:
                continue
            result.extend([d[0] for d in intent[l][-self.num[l]:]])
        return result
        
    def get_article_sentence(self, ar):
        """
        given an article, return the list of sentences
        """
        textList = ar.encode('utf-8').split('。') 
        textList = [t.replace('\n', '') for t in textList]
        textList = filter(lambda x: x != '', textList)
        return textList

    #percent: 取pdf的前多少页
    def extractPDF(self, pdfPath, percent, source = ['table', 'text', 'para']):
        savePath = '/home/zhuyuhe/mydata/jmd/percent/'
        ex = extractor.StockFinanceExtractor(pdfPath, percent)

        nodes = ex.get_pdf_objects(filter_headline=True)

        paraList = []
        textList = []
        textNodes = []
        tableList = []
        tableNodes = []
        senList = []
        for node in nodes:
            if isinstance(node.obj, pdfextractor.pdftree.VirtualNode):
                continue
            if isinstance(node.obj, pdfextractor.page.Table):
                tableNodes.append(node)
            elif  not isinstance(node.obj, pdfextractor.page.Catalog):
                textNodes.append(node)
        if 'table' in source:
            tableList = self.get_node_texts(tableNodes)
            tableList = [t.replace('|', '').split('\n') for t in tableList]
            #二维数组转化为一维
            tableList = sum(tableList,[])
        textList = self.get_node_texts(textNodes)
        if 'para' in source:
            paraList = [t.replace('\n', '') for t in textList]
            paraList = filter(lambda x: x != '', paraList)
        elif 'text' in source:
            senList = sum([t.encode('utf-8').split('。') for t in textList],[])
            senList = [t.replace('\n', '') for t in senList]
            senList = filter(lambda x: x != '', senList)

        pdf = []
    #     pdf.append(textList)
    #     pdf.append(tableList)
    #     pdf = sum(pdf, [])
#         print pdf
        pdf.extend(senList)
#         print 'after extent senList: ' + str(pdf) 
        pdf.extend(tableList)
#         print 'after extent tableList: ' + str(pdf) 
        pdf.extend(paraList)
#         print 'after extent paraList: ' + str(pdf) 
        print 'pdflist length: ' + str(len(pdf))

        return pdf
    
    def get_node_texts(self, pdf_nodes):
        '''
        Given a list of nodes, get the texts of these nodes
        '''
        texts = []
        for node in pdf_nodes:
            text = node.obj.get_text()
            if len(text) <= 25:
                continue
            if u'□' in text or u'√' in text:
                continue
            texts.append(text)
        return texts
    
    def getIntent(self, text):
        """
        return intent of the given text
        result: {'content': __, 'intent': __ , 'prob': __}
        """
        result = {}
        query = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
        predict = query['predict']
        
        result['content'] = text
        result['intent'], result['prob'] = self.intentMap(predict)
        return result
    
    def intentMap(self, predict):
        """
        将现有意图映射到芥末堆所给的意图上去, 并将权重求和
        """
        result = {}
        for l in self.labels:
            result[l] = 0
#         for p in predict:
#             result[self.labelmapR[p['name'].encode('utf-8')]] += p['prob']
#         maxP = max(result.values())
#         intent = result.keys()[result.values().index(maxP)]
#         return intent, maxP
        for key in predict.keys():
            result[self.labelmapR[key.encode('utf-8')]] += predict[key]
        return result
        
        
    def MatchTwoText(self, t1, t2, threshold = 0.5):
        """
        检查两个句子是否匹配
        """
        ct1 = ' '.join(jieba.cut(t1)).split()
        ct2 = ' '.join(jieba.cut(t2)).split()
        commonL = set(ct1).intersection(set(ct2)).__len__()
        if commonL * 1.0 / set(ct1).__len__() > threshold:
            return True
        return False

    def splitByPeriod(self, P):
        """
        split paragraph to sentences
        """
        senList = sum([t.encode('utf-8').split('。') for t in P],[])
        senList = [t.replace('\n', '') for t in senList]
        senList = filter(lambda x: x != '', senList)
        return senList

In [227]:

def getPredict(mode = 'para'):
    for k in jmdata_map.keys():
        text = []
        for p in jmdata_map[k][mode]:
            text.append({'content': p})
        intent = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
        intent = {x['content']: x['features']['intents'] for x in intent}
        for key, value in intent.items():
            intent[key] = {x['name']: x['prob'] for x in value}
            intent[key] = a.intentMap(intent[key])
        jmdata_map[k][mode+'Intent'] = intent
        
def getRec(mode = 'para'):
    num = {}
    labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
    prob = [0.2, 0.3,0.4,0.2,0]
    K = 0
    
    for pdfurl in jmdata_map.keys():
        if mode == 'para':
            K = 0.5 * len(jmdata_map[pdfurl]['content'])
        else:
            K = 1.5 * len(jmdata_map[pdfurl]['content'])
        for i in range(len(labels)):
            num[labels[i]] = (int)(K * prob[i]) + 1
        recList = []
        intent = {}
        for l in labels:
            intent[l] = {}
        for k, v in jmdata_map[pdfurl][mode+'Intent'].items():
            for i, p in v.items():
                intent[i][k] = p
        for i in intent.keys():
            intent[i] = sorted(intent[i].items(), key=operator.itemgetter(1))
            recList.extend([l[0] for l in intent[i][-num[i]:]])
        jmdata_map[pdfurl]['rec'+mode] = list(set(recList))


getPredict('para')
getPredict('sen')

getRec('para')
getRec('sen')

In [228]:
with open('/home/zhuyuhe/mydata/jmd/percent/jmdata_map.pkl', 'wb') as pickle_file:
     pickle.dump(jmdata_map, pickle_file , protocol=2)

In [229]:
a = jmdPDFMap()
P_para_map = []
R_para_map = []
P_sen_map = []
R_sen_map = []
for key in jmdata_map.keys():
    print key  
    print '---------PARA-----------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], recList=jmdata_map[key]['recpara'])
    P_para_map.append(P)
    R_para_map.append(R)
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------SEN-------------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], recList=jmdata_map[key]['recsen'])
    P_sen_map.append(P)
    R_sen_map.append(R)
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '*******************************************************'

http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF
---------PARA-----------
article length: 9
recommend list length: 10
Pcount: 2
Rcount: 3
Pecision: 0.2    Recall: 0.333333333333
---------SEN-------------
article length: 9
recommend list length: 22
Pcount: 3
Rcount: 4
Pecision: 0.136363636364    Recall: 0.444444444444
*******************************************************
http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF
---------PARA-----------
article length: 12
recommend list length: 12
Pcount: 3
Rcount: 4
Pecision: 0.25    Recall: 0.333333333333
---------SEN-------------
article length: 12
recommend list length: 30
Pcount: 4
Rcount: 4
Pecision: 0.133333333333    Recall: 0.333333333333
*******************************************************
http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-15/601801_2016_n.pdf
---------PARA-----------
article length: 7
recommend list length: 9
Pcount: 1
Rcount: 1
Pecision: 0.111111111111    Recall: 0.14285

In [182]:
#结果为-1代表mapcontent长度为0
a = jmdPDFMap()
jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map.pkl', 'rb'))
url = []
for p in jmdata_map.keys():
    if jmdata_map[p]['mapcontent'].__len__() <= 5 and jmdata_map[p]:
        url.append(p)
for u in url:
    del jmdata_map[u]
P_para_map = []
R_para_map = []
P_sen_map = []
R_sen_map = []
for key in jmdata_map.keys():
    print key  
    print '---------PARA-----------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], textList=jmdata_map[key]['para'], mode = 'para')
    P_para_map.append(P)
    R_para_map.append(R)
    jmdata_map[key]['recPara'] = a.getRec()
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------SEN-------------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], textList=jmdata_map[key]['sen'], mode = 'sen')
    P_sen_map.append(P)
    R_sen_map.append(R)
    jmdata_map[key]['recSen'] = a.getRec()
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '*******************************************************'
    

http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF
---------PARA-----------
article length: 9
各个段落（句子）的预测意图分布为：
主营业务$业务进展  46
导语  2
other  72
业绩变动原因  1
未来计划  0
recommend list length: 6
Pcount: 1
Rcount: 3
Pecision: 0.166666666667    Recall: 0.333333333333
---------SEN-------------
article length: 9
各个段落（句子）的预测意图分布为：
主营业务$业务进展  90
导语  3
other  116
业绩变动原因  1
未来计划  1
recommend list length: 11
Pcount: 3
Rcount: 4
Pecision: 0.272727272727    Recall: 0.444444444444
*******************************************************
http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF
---------PARA-----------
article length: 12
各个段落（句子）的预测意图分布为：
主营业务$业务进展  35
导语  4
other  104
业绩变动原因  2
未来计划  0
recommend list length: 7
Pcount: 3
Rcount: 4
Pecision: 0.428571428571    Recall: 0.333333333333
---------SEN-------------
article length: 12
各个段落（句子）的预测意图分布为：
主营业务$业务进展  69
导语  6
other  176
业绩变动原因  3
未来计划  0
recommend list length: 15
Pcount: 2
Rcount: 2
Pecision: 0.133333333333    Recall: 0.16666

In [179]:
text = {'content':'同比下降5%'}
query = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
predict = query[0]['features']['intents']
# prob = [x['prob'] for x in predict]
# i = max(prob)
# intent = predict[prob.index(i)]['name']
# print intent
print query

[{u'content': u'\u540c\u6bd4\u4e0b\u964d5%', u'features': {u'intents': [{u'intent': 0, u'name': u'\u610f\u56fe_\u4f01\u4e1a\u4e1a\u7ee9_\u73b0\u72b6\u504f\u597d', u'prob': 0.002212276922236568}, {u'intent': 1, u'name': u'\u610f\u56fe_\u4f01\u4e1a\u4e1a\u7ee9_\u73b0\u72b6\u504f\u574f', u'prob': 0.8117112713434139}, {u'intent': 2, u'name': u'\u610f\u56fe_\u4f01\u4e1a\u4e1a\u52a1_\u4e1a\u52a1\u524d\u666f', u'prob': 0.0016066750442415175}, {u'intent': 3, u'name': u'\u610f\u56fe_\u4f01\u4e1a\u4e1a\u52a1_\u4e1a\u52a1\u89c4\u5212', u'prob': 0.001476488247944938}, {u'intent': 4, u'name': u'\u610f\u56fe_\u4f01\u4e1a\u4e1a\u52a1_\u4e1a\u52a1\u89c4\u6a21', u'prob': 0.00320301927456518}, {u'intent': 5, u'name': u'\u610f\u56fe_\u4f01\u4e1a\u4e1a\u52a1_\u4e1a\u52a1\u4ecb\u7ecd', u'prob': 0.0007587378795518862}, {u'intent': 6, u'name': u'\u610f\u56fe_\u4f01\u4e1a\u6d3b\u52a8_\u516c\u53f8\u516c\u544a\u4fe1\u606f', u'prob': 0.003704115403952137}, {u'intent': 7, u'name': u'\u610f\u56fe_\u4f01\u4e1a\u6d3

In [230]:
print P_para_map
print R_para_map
print P_sen_map
print R_sen_map

[0.2, 0.25, 0.1111111111111111, 0.36363636363636365, 0.2222222222222222, 0.3, 0.5, 0.3, 0.4166666666666667, 0.16666666666666666, 0.2727272727272727, 0.21428571428571427, 0.2222222222222222, 0.17647058823529413, 0.3333333333333333, 0.16666666666666666, 0.1, 0.3333333333333333, 0.2222222222222222, 0.1111111111111111, 0.42857142857142855, 0.1875, 0.375]
[0.3333333333333333, 0.3333333333333333, 0.14285714285714285, 0.5555555555555556, 0.5714285714285714, 0.8333333333333334, 1.0, 0.375, 0.7777777777777778, 0.2, 0.38461538461538464, 0.42857142857142855, 0.4, 0.18181818181818182, 1.0, 0.375, 0.2222222222222222, 0.8333333333333334, 0.3333333333333333, 0.1111111111111111, 0.5, 0.3125, 0.7]
[0.13636363636363635, 0.13333333333333333, 0.045454545454545456, 0.25, 0.15789473684210525, 0.13636363636363635, 0.30434782608695654, 0.13636363636363635, 0.25925925925925924, 0.0967741935483871, 0.1935483870967742, 0.125, 0.18181818181818182, 0.10204081632653061, 0.2727272727272727, 0.034482758620689655, 0.1

In [231]:
import numpy as np
print 'p_para mean: ' + str(np.mean(filter(lambda x: x!=-1, P_para_map)))
print 'r_para mean: ' + str(np.mean(filter(lambda x: x!=-1, R_para_map)))
print 'p_sen mean: ' + str(np.mean(filter(lambda x: x!=-1, P_sen_map)))
print 'r_sen mean: ' + str(np.mean(filter(lambda x: x!=-1, R_sen_map)))

print "after filter 0:"
print 'p_para mean: ' + str(np.mean(filter(lambda x: x!=0 and x!=-1,P_para_map)))
print 'r_para mean: ' + str(np.mean(filter(lambda x: x!=0 and x!=-1,R_para_map)))
print 'p_sen mean: ' + str(np.mean(filter(lambda x: x!=0 and x!=-1,P_sen_map)))
print 'r_sen mean: ' + str(np.mean(filter(lambda x: x!=0 and x!=-1,R_sen_map)))

p_para mean: 0.259728127087
r_para mean: 0.47413582794
p_sen mean: 0.15187044808
r_sen mean: 0.456872053068
after filter 0:
p_para mean: 0.259728127087
r_para mean: 0.47413582794
p_sen mean: 0.15187044808
r_sen mean: 0.456872053068


In [153]:
u = 'http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF'
print u
for i in jmdata_map[u]['mapcontent']:
    print '-----------'
    print i
print '*************************************'

print 'recommend Paragraph:'
for i in jmdata_map[u]['recPara']:
    print '-----------'
    print i
print '***************************************'

print 'recommend Sentence:'
for i in jmdata_map[u]['recSen']:
    print '------------'
    print i

http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF
-----------
 成立于 2006 年的世纪明德是专业的游学服务提供商，为 3 至 18 岁青少年提供国内游学、国际游学、研学旅行服务，业务涵盖游学产品研发、推广销售、团队运作和后续接待服务等全套流程
-----------
此外，其还面向行业提供教师培训和教育论坛服务
-----------
 报告显示，世纪明德在去年新增国内常规游学线路 14 条，国际游学线路 6 条，师训产品实现了体系化和模块化，客户学校增加了 47.38%
-----------
对于营收等经营指标的增长，报告给出了 3 点原因： 市场环境向好：国家研学旅行政策不断落地，政策环境向好，市场需求大增，客户人数增长迅速； 产品线丰富：加强了国内研学、国际游学、社会实践以及教师培训产品的研发，产品可以覆盖更大客户群，可以为存量客户提供更多的产品服务内容； 客户范围不断扩大：其去年对春秋季研学旅行市场进行了重点突破，从客户参营时间上进行延展，从一年中寒暑假的游学旺季拓展为全年旺季； 从各产品线的营收来看，国内研学产品仍然是世纪明德最重要的业务，营收占比达到 72.33%，收入增长 34.36%
-----------
 世纪明德方面表示，十余年来的积累使得其已经具备了规模优势和亿元级别的销售额，以及近三十万终端客户
-----------
未来，其将利用先发优势，利用投资并购手段，整合部分小型的游学机构
-----------
在业务层面，世纪明德将重点发力国际游学、营地的开发建设以及营地教育，通过设立教育产业并购的方式对行业进行整合
-----------
未来三到五年，世纪明德希望成为中国最大的综合性游学集团
-----------
 报告称，世纪明德在 2017 年计划完成 30 万人次的销售和接待量，实现营业收入持续快速增长
*************************************
recommend Paragraph:
-----------
公司的教师培训业务主要为教育工作者提供学习交流论坛，通过了解客户对于教育培训方面的需求，
-----------
产品研发方面，公司课程研发部门研发百余种研学课程，并汇集成册，形成了“研学旅行在中国”系

In [ ]:
jmdata_mapping = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb'))[2:]
a = jmdPDFMap()
P_para = []
R_para = []
P_sen = []
R_sen = []
i = 0
for jmap in jmdata_mapping:
    print '******************************************************'
    print 'article ' + str(i) + ' :'
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'para')
    P_para.append(P)
    R_para.append(R)
    print "--PARA--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')
    P_sen.append(P)
    R_sen.append(R)
    print "--SEN--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------------------'
    print '*******************************************************'
    i += 1
#print a.getIntent('前三季度利润为0，同比下降5%')['intent']

# jmap = jmd_mapping[0]
# print a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')

In [ ]:
a = jmdPDFMap()
for jmap in jmdata_mapping[14:]:
    print '******************************************************'
    print 'article ' + str(i) + ' :'
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'para')
    P_para.append(P)
    R_para.append(R)
    print "--PARA--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')
    P_sen.append(P)
    R_sen.append(R)
    print "--SEN--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------------------'
    print '*******************************************************'
    i += 1

In [ ]:
print 'P_para: ' + str(P_para)
print 'R_para: ' + str(R_para)
print 'P_sen: ' + str(P_sen)
print 'R_sen: ' + str(R_sen) 
print len(P_para)

In [ ]:
i = 0
jmdata_map = {}
P_para2 = []
P_sen2 = []
R_para2 = []
R_sen2 = []
jmdata_mapping = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb'))[2:]
a = jmdPDFMap()
for jmap in jmdata_mapping:
    key = jmap['pdf_url']
    try:
        para = a.extractPDF(jmap['pdf_url'], 0.5, 'para')
    except BaseException:
        continue
    jmdata_map[key] = {}
    content = a.get_article_sentence(jmap['content'])
    sen = a.splitByPeriod(para)
    jmdata_map[key]['content'] = content
    jmdata_map[key]['para'] = para
    jmdata_map[key]['sen'] = sen
    print '---' + key + ' done' + '----'
    i+=1


Pdfminer Parsed time: 39s
Construct Pdf Tree time: 0s
pdflist length: 182
---http://www.cninfo.com.cn/finalpage/2017-06-20/1203635303.PDF done----
Download pdf time: 7s
download http status code: 200
Pdfminer Parsed time: 36s
Construct Pdf Tree time: 0s
pdflist length: 166
---http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF done----
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 17s
Construct Pdf Tree time: 0s
pdflist length: 46
---http://www.cninfo.com.cn/finalpage/2017-07-26/1203737887.PDF done----
Download pdf time: 6s
download http status code: 200
Pdfminer Parsed time: 35s
Construct Pdf Tree time: 0s
pdflist length: 86
---http://disclosure.szse.cn/finalpage/2017-08-25/1203872097.PDF done----
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 6s
Construct Pdf Tree time: 0s
pdflist length: 11
---http://disclosure.szse.cn/finalpage/2016-10-18/1202765393.PDF done----
Download pdf time: 1s
download http status code: 200
P

In [32]:

print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['content'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['mapcontent'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['para'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['sen'])

18
12
145
255


In [ ]:
Ppara = P_para2
Rpara = R_para2
Psen = P_sen2
R_sen = R_sen2